# Conditional Random Fields
## Named Entity Recognition Tagging

In [1]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

/home/etienne/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/etienne/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Here we take words read from the input files and construct features we use to train the CRF.

For every input word we create features for the lower case verison of the word, the last 2 and the last 3 characters of the word, whether the word is in upper case, title case or whether it consists of digits. We also create features for the previous word and the next word in the sentence.

In [2]:
def word2features(sent, i):
    word = sent[i][0]

    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit()
    ]
    if i > 0:
        word1 = sent[i-1][0]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper()
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper()
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [postag for token, postag in sent]

def sent2tokens(sent):
    return [token for token, postag in sent]

Here we open our training and testing files, parse them and pass them to the feature extraction functions to ready them for training.

In [10]:
def getSentsFromFile(file):
    #all_sents = []

    this_sent = []

    for line in file:
        line= line.strip()
        if line=="":
            #all_sents.extend([this_sent])
            yield(this_sent)
            this_sent=[]
        
            continue
        else:
            parts = line.split("\t")
            this_sent.extend([tuple(parts)])
      
    return


def loadTestTrain(trainFile, testFile):
    train_sents = [s for s in getSentsFromFile(trainFile)]
    test_sents = [s for s in getSentsFromFile(testFile)]

    X_train = [sent2features(s) for s in train_sents]
    y_train = [sent2labels(s) for s in train_sents]

    X_test = [sent2features(s) for s in test_sents]
    y_test = [sent2labels(s) for s in test_sents]

    print "Loaded ", len(X_train), " training instances/", len(X_test), " testing instances."
    
    return (X_train, y_train, X_test, y_test)

trainf_ner = open("Data/ner-pol/train.iob")
testf_ner = open("Data/ner-pol/test.iob")

(X_train, y_train, X_test, y_test) = loadTestTrain(trainf_ner, testf_ner)


Loaded  14987  training instances/ 3684  testing instances.


Set up model hyperparameters and train the model.

In [28]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=1.0,
    c2=0.001,
    max_iterations=50,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 25.1 s, sys: 230 ms, total: 25.3 s
Wall time: 25.6 s


List the model labels consisting of named entity types - Organisation, person, location and miscellaneous.

In [29]:
labels = list(crf.classes_)
labels

['|O',
 '|B-ORG',
 '|B-MISC',
 '|B-PER',
 '|I-PER',
 '|B-LOC',
 '|I-ORG',
 '|I-MISC',
 '|I-LOC']

Process the testing set and print out the f1-score.

In [33]:
%%time
y_pred = crf.predict(X_test)
print metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9497753323961592
CPU times: user 1.18 s, sys: 20 µs, total: 1.18 s
Wall time: 1.17 s


Print out prediction costs per label

In [31]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

     |B-LOC      0.846     0.785     0.814      1669
    |B-MISC      0.806     0.740     0.772       705
     |B-ORG      0.798     0.667     0.727      1663
     |B-PER      0.790     0.843     0.816      1617
     |I-LOC      0.736     0.578     0.648       256
    |I-MISC      0.634     0.643     0.639       213
     |I-ORG      0.653     0.701     0.676       833
     |I-PER      0.838     0.948     0.890      1156
         |O      0.983     0.988     0.986     38554

avg / total      0.950     0.951     0.950     46666



List the features by training importance

In [37]:
for key, value in sorted(crf.state_features_.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    print "%s: %s" % (key, value)
    

(u'word[-2:]=5M', u'|O'): 7.942236
(u'word[-2:]=0M', u'|O'): 7.584323
(u'-1:word.lower=grade', u'|O'): 7.467747
(u'word.lower=ata-ur-rehman', u'|B-PER'): 7.377029
(u'word.lower=sungard', u'|B-ORG'): 6.664536
(u'word[-2:]=6F', u'|O'): 6.550478
(u'word.lower=minister', u'|O'): 6.19998
(u'+1:word.lower=1996-08-26', u'|B-LOC'): 5.880611
(u'word.lower=serb-held', u'|B-MISC'): 5.853415
(u'word.lower=weston-super-mare', u'|B-LOC'): 5.847896
(u'word.lower=mcgrath', u'|B-PER'): 5.788491
(u'word[-2:]=-A', u'|O'): 5.677977
(u'+1:word.lower=1996-08-23', u'|B-LOC'): 5.531767
(u'word.lower=england', u'|B-LOC'): 5.348388
(u'word.lower=mclaren', u'|B-ORG'): 5.332102
(u'word.lower=parma', u'|B-ORG'): 5.327616
(u'+1:word.lower=1996-08-28', u'|B-LOC'): 5.295032
(u'word.lower=german', u'|B-MISC'): 5.289305
(u'+1:word.lower=1996-08-27', u'|B-LOC'): 5.262556
(u'+1:word.lower=1996-08-29', u'|B-LOC'): 5.262069
(u'+1:word.lower=1996-08-25', u'|B-LOC'): 5.257143
(u'word.lower=clinton', u'|B-PER'): 5.256348
(u'w

(u'+1:word.lower=2', u'|B-ORG'): 2.877508
(u'word[-3:]=hat', u'|O'): 2.876008
(u'-1:word.lower=at', u'|B-LOC'): 2.872825
(u'-1:word.lower=flight', u'|B-MISC'): 2.869963
(u'word.lower=neuchatel', u'|B-ORG'): 2.867003
(u'word.lower=lelouche', u'|B-PER'): 2.865053
(u'word.lower=bastia', u'|B-ORG'): 2.865025
(u'word.lower=nantes', u'|B-ORG'): 2.862773
(u'word[-2:]=45', u'|O'): 2.859702
(u'word[-3:]=H1', u'|O'): 2.855025
(u'word[-2:]=H1', u'|O'): 2.855025
(u'word.lower=h1', u'|O'): 2.855025
(u'word[-2:]=OJ', u'|B-ORG'): 2.854981
(u'word.lower=as', u'|O'): 2.854788
(u'word[-3:]=cor', u'|B-ORG'): 2.851842
(u'word.lower=pivotal', u'|B-PER'): 2.847795
(u'word.lower=vanlandingham', u'|B-PER'): 2.845799
(u'word.lower=when', u'|O'): 2.83011
(u'word[-3:]=C', u'|O'): 2.829701
(u'word[-2:]=C', u'|O'): 2.829701
(u'word.lower=bonds', u'|B-PER'): 2.828594
(u'word.lower=classic', u'|I-MISC'): 2.828532
(u'+1:word.lower=cup', u'|B-MISC'): 2.828077
(u'word.lower=israeli', u'|B-MISC'): 2.822062
(u'word.lower

(u'word[-3:]=ath', u'|I-PER'): 1.28553
(u'-1:word.lower=compatriot', u'|B-PER'): 1.285296
(u'-1:word.lower=banco', u'|I-ORG'): 1.285011
(u'word.lower=federation', u'|I-LOC'): 1.283645
(u'word[-3:]=ans', u'|I-MISC'): 1.282667
(u'word.lower=you', u'|O'): 1.282644
(u'word[-2:]=ga', u'|I-PER'): 1.282304
(u'word[-3:]=TAN', u'|B-LOC'): 1.281565
(u'word[-3:]=ary', u'|I-LOC'): 1.281064
(u'word.lower=gente', u'|B-ORG'): 1.280892
(u'word.lower=federal', u'|B-ORG'): 1.280699
(u'word[-3:]=ort', u'|I-MISC'): 1.280601
(u'-1:word.lower=fred', u'|I-PER'): 1.279168
(u'word.lower=hong', u'|B-MISC'): 1.278212
(u'word[-3:]=pic', u'|B-MISC'): 1.277546
(u'word[-3:]=ND', u'|B-ORG'): 1.277399
(u'word.lower=nd', u'|B-ORG'): 1.277399
(u'-1:word.lower=1', u'|B-ORG'): 1.276895
(u'word.lower=captain', u'|O'): 1.275415
(u'word[-3:]=ela', u'|B-LOC'): 1.274952
(u'word.lower=kornblum', u'|B-PER'): 1.274836
(u'word[-3:]=ann', u'|B-ORG'): 1.273793
(u'+1:word.lower=states', u'|B-LOC'): 1.273438
(u'+1:word.lower=bus', u'|

(u'-1:word.lower=swiss', u'|B-MISC'): 0.758219
(u'+1:word.lower=win', u'|B-ORG'): 0.758136
(u'word[-2:]=AT', u'|O'): 0.758082
(u'+1:word.lower=television', u'|B-ORG'): 0.757279
(u'-1:word.lower=bosnian', u'|I-MISC'): 0.757054
(u"+1:word.lower='", u'|B-PER'): 0.756231
(u'word.lower=of', u'|I-ORG'): 0.756099
(u'word[-2:]=my', u'|I-ORG'): 0.755689
(u'word[-3:]=ley', u'|B-PER'): 0.755497
(u'word[-3:]=tes', u'|I-LOC'): 0.755409
(u'-1:word.lower=of', u'|I-ORG'): 0.755337
(u'+1:word.lower=kingdom', u'|B-LOC'): 0.755237
(u'word[-3:]=PEX', u'|B-ORG'): 0.755083
(u'word.lower=pulpex', u'|B-ORG'): 0.755083
(u'-1:word.lower=vs.', u'|B-PER'): 0.755067
(u'word.lower=masood', u'|B-PER'): 0.754759
(u'word.lower=free', u'|I-LOC'): 0.754211
(u'word.lower=its', u'|O'): 0.753554
(u'-1:word.lower=sudan', u'|I-ORG'): 0.753296
(u'+1:word.lower=rights', u'|O'): 0.753116
(u'-1:word.lower=main', u'|B-MISC'): 0.753105
(u'word.lower=assembly', u'|I-ORG'): 0.752989
(u'word.lower=taibe', u'|B-ORG'): 0.752267
(u'word

(u'-1:word.lower=ltd', u'|O'): 0.391282
(u'word.lower=suspected', u'|O'): 0.390917
(u'+1:word.lower=government', u'|B-MISC'): 0.390858
(u'+1:word.lower=bank', u'|I-ORG'): 0.390848
(u'+1:word.isupper=True', u'|B-MISC'): 0.39074
(u'word[-2:]=ve', u'|O'): 0.390479
(u'word[-3:]=zon', u'|B-LOC'): 0.390272
(u'-1:word.lower=accused', u'|B-LOC'): 0.389383
(u'word[-3:]=igh', u'|B-LOC'): 0.389138
(u'word.lower=motorola', u'|B-ORG'): 0.389062
(u'word.lower=jamaica', u'|B-LOC'): 0.389029
(u'word[-2:]=LI', u'|B-LOC'): 0.388428
(u'-1:word.lower=state', u'|I-ORG'): 0.38839
(u'word[-3:]=mad', u'|B-PER'): 0.387642
(u'word[-2:]=OF', u'|O'): 0.387625
(u'+1:word.lower=sale', u'|O'): 0.387284
(u'word[-3:]=ria', u'|B-ORG'): 0.38728
(u'word[-2:]=te', u'|O'): 0.386623
(u'word.lower=tyrrell', u'|B-ORG'): 0.386199
(u'-1:word.lower=bold', u'|I-ORG'): 0.386057
(u'+1:word.lower=ogilvy', u'|I-ORG'): 0.386057
(u'-1:word.lower=african', u'|I-LOC'): 0.385841
(u'word.lower=fort', u'|B-LOC'): 0.385787
(u'-1:word.lower=s

(u'word[-2:]=et', u'|I-ORG'): 0.128505
(u'word[-3:]=Ken', u'|B-PER'): 0.12819
(u'word.lower=ken', u'|B-PER'): 0.12819
(u'word[-3:]=MLN', u'|O'): 0.128094
(u'word[-2:]=LN', u'|O'): 0.128094
(u'word[-2:]=AL', u'|B-ORG'): 0.127845
(u'word[-2:]=ie', u'|I-PER'): 0.127831
(u'word[-3:]=Sun', u'|I-ORG'): 0.127586
(u'word.lower=sun', u'|I-ORG'): 0.127586
(u'+1:word.lower=parma', u'|O'): 0.127538
(u'word.lower=stocks', u'|O'): 0.127288
(u'-1:word.lower=replace', u'|B-PER'): 0.127259
(u'-1:word.lower=prince', u'|B-PER'): 0.127198
(u'word.lower=cork', u'|I-PER'): 0.127064
(u'+1:word.lower=financial', u'|B-ORG'): 0.12629
(u'word[-2:]=NG', u'|I-LOC'): 0.126287
(u'-1:word.lower=by', u'|B-PER'): 0.126137
(u'word.lower=el', u'|B-ORG'): 0.125987
(u'word[-2:]=fe', u'|I-ORG'): 0.125923
(u'-1:word.lower=japanese', u'|B-ORG'): 0.125748
(u'word.lower=recomposed', u'|O'): 0.125739
(u'word[-3:]=Act', u'|I-MISC'): 0.12553
(u'word[-2:]=ue', u'|I-MISC'): 0.125485
(u'word[-3:]=ulf', u'|B-MISC'): 0.125338
(u'word.l

(u'+1:word.lower=rafferty', u'|B-PER'): 0.015001
(u'word.lower=practice', u'|O'): 0.014982
(u'word[-3:]=ner', u'|B-PER'): 0.014971
(u'word[-2:]=se', u'|B-PER'): 0.01486
(u'+1:word.lower=grade', u'|B-MISC'): 0.014846
(u'word[-2:]=ft', u'|I-PER'): 0.014831
(u'word[-3:]=(', u'|I-LOC'): 0.014824
(u'word[-2:]=(', u'|I-LOC'): 0.014824
(u'word.lower=(', u'|I-LOC'): 0.014824
(u'word[-3:]=ORK', u'|B-LOC'): 0.014804
(u'word.lower=prince', u'|B-PER'): 0.014804
(u'word.lower=national', u'|B-ORG'): 0.014771
(u'word[-3:]=nts', u'|I-ORG'): 0.014689
(u'+1:word.lower=front', u'|I-ORG'): 0.014612
(u'word[-2:]=RK', u'|B-LOC'): 0.014525
(u'word[-3:]=val', u'|I-MISC'): 0.014427
(u'word[-3:]=tie', u'|I-PER'): 0.014362
(u'word.lower=motor', u'|I-ORG'): 0.014233
(u'+1:word.lower=co', u'|B-ORG'): 0.014233
(u'word.lower=saskatchewan', u'|B-LOC'): 0.014209
(u'word[-3:]=aun', u'|B-PER'): 0.014182
(u'word.lower=shaun', u'|B-PER'): 0.014182
(u'word.lower=international', u'|O'): 0.014181
(u'word[-2:]=ne', u'|B-LOC')

(u'+1:word.lower=6', u'|B-LOC'): -0.525479
(u"+1:word.lower='s", u'|I-MISC'): -0.525604
(u'word[-3:]=ard', u'|O'): -0.525796
(u'+1:word.lower=on', u'|B-PER'): -0.527571
(u'word[-3:]=ine', u'|B-ORG'): -0.52974
(u'+1:word.lower=district', u'|I-ORG'): -0.530596
(u'+1:word.lower=was', u'|B-MISC'): -0.532027
(u'word.lower=st', u'|O'): -0.53211
(u'-1:word.lower=to', u'|O'): -0.535546
(u'+1:word.lower=.', u'|B-MISC'): -0.538742
(u'word.lower=washington', u'|B-ORG'): -0.540562
(u'+1:word.lower=newspaper', u'|O'): -0.541366
(u'word[-2:]=ON', u'|O'): -0.544114
(u'word.lower=west', u'|B-ORG'): -0.548008
(u'+1:word.lower=official', u'|O'): -0.549462
(u'+1:word.lower=7', u'|I-PER'): -0.550387
(u'word[-3:]=EAL', u'|O'): -0.556689
(u'-1:word.lower=and', u'|B-MISC'): -0.557212
(u'word[-3:]=AND', u'|O'): -0.558548
(u'word[-2:]=ni', u'|B-LOC'): -0.559219
(u'bias', u'|I-PER'): -0.559863
(u'word[-3:]=art', u'|O'): -0.560881
(u'word[-2:]=ru', u'|O'): -0.562863
(u'word.lower=city', u'|O'): -0.563414
(u'word

Set up a grid search and 3-fold cross-validation.

This will take HOURS to run. Note that max_iterations has been decreased to 20 (from 50) in order to expedite the task. This however causes a decrease in prediction performance compared to the previous cells.

In [19]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=20,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 82.9min finished


CPU times: user 1h 21min 32s, sys: 1min 1s, total: 1h 22min 33s
Wall time: 1h 23min 7s


Inspect the scores from the grid search. Note that the f1-score doesn't vary very much with the c1/c2 parameters and is more dependent on max_iterations.

In [38]:
rs.grid_scores_

[mean: 0.93933, std: 0.00517, params: {'c2': 0.05224537719245085, 'c1': 0.1552505912922396},
 mean: 0.93920, std: 0.00486, params: {'c2': 0.014664972677450788, 'c1': 0.2181638197432444},
 mean: 0.93563, std: 0.00529, params: {'c2': 0.058159123627147515, 'c1': 0.4849499133987302},
 mean: 0.93755, std: 0.00339, params: {'c2': 0.05878669973066139, 'c1': 0.28935072323291616},
 mean: 0.93915, std: 0.00312, params: {'c2': 0.027818150565818295, 'c1': 0.5510471061891358},
 mean: 0.93560, std: 0.00508, params: {'c2': 0.08667614497770654, 'c1': 0.40032222000802303},
 mean: 0.93943, std: 0.00361, params: {'c2': 0.08686049015897769, 'c1': 0.5662766651417782},
 mean: 0.93858, std: 0.00452, params: {'c2': 0.2121673953171459, 'c1': 0.41193448415135336},
 mean: 0.94031, std: 0.00454, params: {'c2': 0.04348195360013139, 'c1': 0.06138733444418406},
 mean: 0.93568, std: 0.00357, params: {'c2': 0.003559452214544051, 'c1': 1.21303629583201},
 mean: 0.93579, std: 0.00539, params: {'c2': 0.06606988307597234,

In [20]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

('best params:', {'c2': 0.025601748972799855, 'c1': 0.11303458841011417})
('best CV score:', 0.9406488963420524)
model size: 3.00M


Print the per-label scores and the features ordered by importance for the best model found by the grid search.

In [27]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

for key, value in sorted(crf.state_features_.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    print "%s: %s" % (key, value)


             precision    recall  f1-score   support

     |B-LOC      0.728     0.664     0.694      1669
    |B-MISC      0.654     0.671     0.662       705
     |B-ORG      0.746     0.435     0.549      1663
     |B-PER      0.728     0.776     0.751      1617
     |I-LOC      0.592     0.492     0.537       256
    |I-MISC      0.236     0.671     0.349       213
     |I-ORG      0.688     0.517     0.591       833
     |I-PER      0.787     0.912     0.845      1156
         |O      0.971     0.981     0.976     38554

avg / total      0.926     0.924     0.922     46666

(u'word[-3:]=I', u'|O'): 4.600259
(u'word[-2:]=I', u'|O'): 4.600259
(u'word.lower=i', u'|O'): 4.583866
(u'word.lower=a', u'|O'): 4.203804
(u'-1:word.lower=at', u'|B-LOC'): 3.583158
(u'word.isupper=False', u'|O'): 3.145201
(u'word[-3:]=A', u'|O'): 3.115499
(u'word[-2:]=A', u'|O'): 3.115499
(u'word.istitle=False', u'|O'): 3.102066
(u'+1:word.lower=3', u'|B-ORG'): 2.800347
(u'word[-2:]=T-', u'|O'): 2.755228
(u'wor

(u'word[-2:]=SH', u'|B-MISC'): 0.559365
(u'+1:word.lower=newsroom', u'|B-LOC'): 0.557292
(u'word.lower=city', u'|I-LOC'): 0.556958
(u'+1:word.lower=union', u'|B-ORG'): 0.555021
(u'+1:word.lower=)', u'|I-LOC'): 0.552779
(u'word.lower=co', u'|I-ORG'): 0.549524
(u'word[-2:]=TA', u'|B-ORG'): 0.547297
(u'-1:word.lower=an', u'|B-MISC'): 0.547079
(u'word.lower=this', u'|O'): 0.546702
(u'word[-2:]=ex', u'|B-ORG'): 0.545964
(u'word.lower=was', u'|O'): 0.544914
(u'word[-3:]=ole', u'|B-PER'): 0.544332
(u'word.lower=chicago', u'|B-ORG'): 0.542777
(u'word[-3:]=sin', u'|B-PER'): 0.542038
(u'word[-3:]=ate', u'|O'): 0.539482
(u'word[-3:]=nds', u'|B-LOC'): 0.538089
(u'word[-3:]=rld', u'|B-MISC'): 0.537939
(u'word[-3:]=was', u'|O'): 0.537187
(u'+1:word.lower=government', u'|B-MISC'): 0.536883
(u'word[-3:]=May', u'|O'): 0.535262
(u'word[-3:]=ign', u'|O'): 0.534248
(u'word[-2:]=gn', u'|O'): 0.534248
(u'word[-3:]=men', u'|O'): 0.534065
(u'word.lower=arafat', u'|B-PER'): 0.533588
(u'word[-3:]=St', u'|B-ORG'

(u'word[-3:]=BSE', u'|B-MISC'): 0.180853
(u'word.lower=bse', u'|B-MISC'): 0.180853
(u'-1:word.lower=jim', u'|I-PER'): 0.180648
(u'word[-2:]=nd', u'|I-LOC'): 0.180637
(u'word[-2:]=EY', u'|B-LOC'): 0.180632
(u'word[-3:]=ker', u'|I-PER'): 0.180606
(u'word.lower=day', u'|I-MISC'): 0.180285
(u'word[-3:]=Red', u'|B-ORG'): 0.18014
(u'-1:word.lower=pakistan', u'|O'): 0.180123
(u'word[-3:]=C.', u'|B-PER'): 0.180079
(u'word.lower=c.', u'|B-PER'): 0.180079
(u'word[-2:]=OF', u'|O'): 0.179978
(u'word[-3:]=kee', u'|B-ORG'): 0.179893
(u'word.lower=cairo', u'|B-LOC'): 0.179819
(u'+1:word.lower=index', u'|I-MISC'): 0.1798
(u'word.lower=fiorentina', u'|B-ORG'): 0.179597
(u'word[-3:]=OB', u'|B-ORG'): 0.17959
(u'word.lower=ob', u'|B-ORG'): 0.17959
(u'word[-2:]=FK', u'|B-ORG'): 0.179564
(u'+1:word.lower=95', u'|B-MISC'): 0.179554
(u'word[-3:]=OUL', u'|B-LOC'): 0.179463
(u'word[-3:]=ose', u'|B-PER'): 0.17931
(u'word[-3:]=nte', u'|B-ORG'): 0.1793
(u'+1:word.lower=three', u'|O'): 0.179139
(u'word.lower=singap

(u'+1:word.lower=27', u'|O'): 0.101731
(u'word[-3:]=ria', u'|B-ORG'): 0.101727
(u'word[-3:]=Gov', u'|O'): 0.101687
(u'word[-2:]=ou', u'|O'): 0.101667
(u'+1:word.lower=sept', u'|O'): 0.101596
(u'-1:word.lower=human', u'|I-ORG'): 0.101576
(u'word[-3:]=lla', u'|I-PER'): 0.101573
(u'word.lower=islamabad', u'|B-LOC'): 0.101521
(u'-1:word.lower=one', u'|B-MISC'): 0.101469
(u'word[-3:]=VfB', u'|B-ORG'): 0.101447
(u'word[-2:]=fB', u'|B-ORG'): 0.101447
(u'word.lower=vfb', u'|B-ORG'): 0.101447
(u'word.lower=commissioner', u'|O'): 0.101425
(u'word.lower=janeiro', u'|I-LOC'): 0.101378
(u'word[-3:]=rry', u'|I-PER'): 0.101367
(u'word[-3:]=ope', u'|I-ORG'): 0.101363
(u'word.lower=huddersfield', u'|B-ORG'): 0.101266
(u'+1:word.lower=prix', u'|I-MISC'): 0.101249
(u'word[-3:]=96', u'|I-MISC'): 0.10124
(u'word.lower=96', u'|I-MISC'): 0.10124
(u'word[-2:]=RO', u'|B-LOC'): 0.101231
(u'word.lower=plymouth', u'|B-ORG'): 0.101197
(u'+1:word.lower=stuttgart', u'|B-ORG'): 0.101175
(u'-1:word.lower=rio', u'|I-LO

(u'word[-3:]=kan', u'|B-ORG'): 0.068393
(u'+1:word.lower=ptt', u'|B-ORG'): 0.068377
(u'word[-3:]=ard', u'|B-MISC'): 0.068346
(u'+1:word.lower=42', u'|B-ORG'): 0.068345
(u'-1:word.lower=ieng', u'|I-PER'): 0.06831
(u'word[-3:]=INS', u'|O'): 0.068299
(u'word[-3:]=St.', u'|B-LOC'): 0.068261
(u'word.lower=st.', u'|B-LOC'): 0.068261
(u'-1:word.lower=gazeta', u'|I-ORG'): 0.068244
(u'word.lower=despite', u'|O'): 0.068234
(u'word[-2:]=14', u'|O'): 0.068166
(u'+1:word.lower=1-1', u'|O'): 0.068166
(u'word.lower=puchon', u'|B-ORG'): 0.068114
(u'+1:word.lower=sri', u'|O'): 0.068083
(u'word.lower=kashmir', u'|B-LOC'): 0.068033
(u'word[-3:]=13', u'|O'): 0.067986
(u'word.lower=13', u'|O'): 0.067986
(u'word[-3:]=NIS', u'|O'): 0.067942
(u'word.lower=municipal', u'|I-ORG'): 0.06793
(u'word[-3:]=nin', u'|B-PER'): 0.067927
(u'word[-2:]=AK', u'|B-ORG'): 0.067914
(u'word[-3:]=All', u'|B-ORG'): 0.067906
(u'-1:word.lower=frenchman', u'|B-PER'): 0.067871
(u'+1:word.lower=announcement', u'|O'): 0.067822
(u'word[

...

(u'word[-3:]=ler', u'|B-MISC'): -0.001158
(u'-1:word.lower=dacom', u'|O'): -0.001158
(u'word[-2:]=bo', u'|I-PER'): -0.001163
(u'word[-3:]=ult', u'|I-PER'): -0.001164
(u'+1:word.lower=bridge', u'|B-ORG'): -0.001164
(u'-1:word.lower=julie', u'|O'): -0.001166
(u'-1:word.lower=accept', u'|O'): -0.001175
(u'word[-2:]=ap', u'|B-LOC'): -0.001177
(u'word.lower=white', u'|I-ORG'): -0.001177
(u'word.lower=jerusalem', u'|I-ORG'): -0.001177
(u'word[-3:]=oyd', u'|B-PER'): -0.001178
(u'word[-2:]=yd', u'|B-PER'): -0.001178
(u'word.lower=lloyd', u'|B-PER'): -0.001178
(u'-1:word.lower=gruppen', u'|O'): -0.001178
(u'word[-2:]=em', u'|I-MISC'): -0.001181
(u'-1:word.lower=sport', u'|O'): -0.001185
(u'+1:word.lower=lewis', u'|O'): -0.001188
(u'word[-3:]=rch', u'|I-PER'): -0.00119
(u'+1:word.lower=team', u'|I-ORG'): -0.001191
(u'word.lower=council', u'|B-MISC'): -0.001192
(u'word[-3:]=La', u'|I-ORG'): -0.001194
(u'+1:word.lower=j.', u'|O'): -0.001194
(u'-1:word.lower=systems', u'|O'): -0.001198
(u'+1:word.l

(u'+1:word.lower=nt', u'|O'): -0.004214
(u'+1:word.lower=100-30', u'|O'): -0.004215
(u'-1:word.lower=egyptair', u'|O'): -0.004217
(u'+1:word.lower=kick', u'|O'): -0.004217
(u'-1:word.lower=refuse', u'|O'): -0.004218
(u'-1:word.lower=dictator', u'|O'): -0.004224
(u'word[-3:]=yal', u'|O'): -0.004227
(u'-1:word.lower=brutal', u'|O'): -0.004227
(u'+1:word.lower=1860', u'|O'): -0.004227
(u'-1:word.lower=communist', u'|B-PER'): -0.004231
(u'-1:word.lower=repaid', u'|O'): -0.004235
(u'-1:word.lower=1997-98', u'|O'): -0.004236
(u'word[-3:]=Max', u'|O'): -0.004246
(u'word[-3:]=Fox', u'|B-ORG'): -0.004246
(u'word.lower=max', u'|O'): -0.004246
(u'word.lower=fox', u'|B-ORG'): -0.004246
(u'-1:word.lower=scoring', u'|O'): -0.004249
(u'+1:word.lower=missionaries', u'|O'): -0.004249
(u'-1:word.lower=fee', u'|O'): -0.004257
(u'+1:word.lower=152', u'|O'): -0.004261
(u'-1:word.lower=slammed', u'|O'): -0.004269
(u'-1:word.lower=utility', u'|O'): -0.00427
(u'-1:word.lower=checkfree', u'|O'): -0.00427
(u'+1

(u'+1:word.lower=partners', u'|O'): -0.014084
(u'word.lower=university', u'|O'): -0.014121
(u'word.lower=rapid', u'|O'): -0.014128
(u'+1:word.lower=laying', u'|O'): -0.01415
(u'+1:word.lower=9', u'|I-PER'): -0.014155
(u'word[-2:]=cs', u'|O'): -0.014161
(u'word[-3:]=les', u'|O'): -0.014167
(u'word[-3:]=04', u'|O'): -0.014167
(u'word.lower=villa', u'|I-PER'): -0.014167
(u'word.lower=04', u'|O'): -0.014167
(u'-1:word.lower=typical', u'|O'): -0.014167
(u'+1:word.lower=interbank', u'|O'): -0.014179
(u'word[-2:]=ie', u'|I-LOC'): -0.014195
(u'-1:word.lower=democracy', u'|O'): -0.014196
(u'word[-3:]=bor', u'|B-ORG'): -0.014197
(u'+1:word.lower=pitched', u'|O'): -0.0142
(u'-1:word.lower=george', u'|O'): -0.014206
(u'word[-3:]=kan', u'|O'): -0.014209
(u'word[-3:]=ash', u'|B-LOC'): -0.014218
(u'+1:word.lower=marketing', u'|O'): -0.01422
(u'word[-2:]=ES', u'|B-PER'): -0.014224
(u'word[-3:]=ung', u'|B-LOC'): -0.014235
(u'+1:word.lower=hovercrafts', u'|O'): -0.014236
(u'+1:word.lower=1970s', u'|O'):

(u'-1:word.lower=nations', u'|O'): -0.046492
(u'word[-3:]=lim', u'|O'): -0.046502
(u'word.lower=park', u'|O'): -0.046537
(u'word[-2:]=ff', u'|I-ORG'): -0.046597
(u'+1:word.lower=farmers', u'|O'): -0.046611
(u'word.lower=state', u'|B-LOC'): -0.046648
(u'+1:word.lower=22', u'|B-PER'): -0.046672
(u'+1:word.lower=4-0', u'|O'): -0.046786
(u'+1:word.lower=unless', u'|O'): -0.046868
(u'+1:word.lower=morinville', u'|O'): -0.046899
(u'word.isupper=True', u'|I-ORG'): -0.046977
(u'word[-2:]=rn', u'|I-ORG'): -0.04705
(u'word[-2:]=se', u'|I-PER'): -0.047191
(u'word[-3:]=urn', u'|O'): -0.047246
(u'word.lower=the', u'|I-LOC'): -0.047269
(u'word[-3:]=ane', u'|O'): -0.047297
(u'+1:word.lower=but', u'|B-ORG'): -0.04733
(u'word.lower=union', u'|B-ORG'): -0.047362
(u'+1:word.lower=militants', u'|O'): -0.047383
(u'word.lower=york', u'|B-ORG'): -0.047413
(u'+1:word.lower=two', u'|B-LOC'): -0.047418
(u'word.lower=bill', u'|O'): -0.047466
(u'word.lower=taiwan', u'|B-ORG'): -0.047475
(u'+1:word.lower=1996-08-2

(u'word[-3:]=her', u'|B-LOC'): -0.177379
(u'-1:word.lower=capital', u'|O'): -0.177886
(u'word[-3:]=ard', u'|B-LOC'): -0.177993
(u'word[-2:]=um', u'|B-ORG'): -0.17824
(u'word[-2:]=me', u'|I-ORG'): -0.178848
(u'word[-3:]=bed', u'|O'): -0.178861
(u'word[-3:]=sex', u'|O'): -0.179049
(u'word[-2:]=ns', u'|O'): -0.179585
(u'+1:word.istitle=True', u'|I-PER'): -0.179946
(u'word[-3:]=ice', u'|B-PER'): -0.180011
(u'word[-3:]=ing', u'|I-LOC'): -0.18011
(u'word[-2:]=ex', u'|B-LOC'): -0.180565
(u'word[-3:]=rch', u'|B-LOC'): -0.181112
(u'word[-3:]=RAL', u'|O'): -0.181326
(u'word[-3:]=nia', u'|B-MISC'): -0.181555
(u'-1:word.lower=former', u'|B-PER'): -0.181704
(u'word[-3:]=ity', u'|B-LOC'): -0.181808
(u'word.lower=at', u'|I-ORG'): -0.181846
(u'word[-3:]=can', u'|I-ORG'): -0.182464
(u'word[-2:]=ka', u'|I-PER'): -0.182562
(u'word[-3:]=mas', u'|O'): -0.182668
(u'word[-3:]=rry', u'|O'): -0.18274
(u'word[-3:]=ISS', u'|O'): -0.183407
(u'-1:word.lower="', u'|B-MISC'): -0.18344
(u'word.lower=west', u'|O'): -0

Save the trained model to disk.

In [23]:
import pickle

f = open("crf_ner_pol.mdl", "a")
pickle.dump(crf, f)
f.close()

Print the most likely and least likely transitions.

In [24]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
|B-LOC -> |I-LOC  5.345479
|B-PER -> |I-PER  5.174143
|I-ORG -> |I-ORG  4.609703
|I-MISC -> |I-MISC 4.569016
|B-ORG -> |I-ORG  4.533346
|B-MISC -> |I-MISC 3.129953
|I-PER -> |I-PER  1.794438
|O     -> |B-ORG  1.580775
|O     -> |B-LOC  1.509977
|O     -> |B-MISC 1.488459
|I-LOC -> |I-LOC  1.441020
|O     -> |B-PER  1.336434
|O     -> |O      0.822066
|B-LOC -> |O      0.562223
|B-PER -> |O      0.368204
|I-PER -> |O      0.046463
|B-MISC -> |B-MISC 0.024453
|B-MISC -> |O      0.021723
|B-MISC -> |B-PER  -0.068634
|B-LOC -> |B-MISC -0.095446

Top unlikely transitions:
|I-ORG -> |B-PER  -1.766387
|B-PER -> |B-LOC  -1.771607
|B-LOC -> |B-PER  -1.813408
|B-PER -> |B-MISC -1.937413
|B-MISC -> |I-PER  -2.015148
|B-MISC -> |I-ORG  -2.102599
|I-PER -> |I-ORG  -2.139164
|I-ORG -> |I-PER  -2.330169
|B-LOC -> |I-ORG  -2.514656
|B-ORG -> |B-PER  -2.631615
|B-ORG -> |B-LOC  -2.706786
|B-LOC -> |I-PER  -2.823604
|B-PER -> |B-ORG  -3.407979
|B-ORG -> |I-PER  -3.809023
|B-PER -